In [59]:
import pandas as pd
df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/sythesize_from_scratch/hospworks_data.csv")
df.head()

,interaction_id,user_id,event_id,interaction_type,interaction_time,interaction_distance_to_event,users_user_lat,users_user_lon,users_user_city,users_user_weather_preference,...,events_event_lat,events_event_lon,events_event_city,events_start_time,events_duration,events_weather_condition,events_temperature,events_attendance_rate,events_event_indoor_capability,interaction_label
0,0794ae31-b867-4bd8-84f3-763cdcbb81d6,NW814V,ZG645S,no,2023-06-13 06:55:37.728541+00:00,5810.000000,51.409589,-0.109415,London,outdoor,...,42.145335,-78.925101,Toronto,2025-03-30 10:06:51.523437+00:00,240,Rain,9.1,83.238042,True,0
1,fa403386-53fc-4f68-8fcf-6791931bde80,UT633Z,RE468E,no,2024-07-14 07:19:15.832112+00:00,7785.000000,35.598507,139.616940,Tokyo,indoor,...,-33.875733,151.218054,Sydney,2025-08-14 19:26:31.045513+00:00,180,Clear,26.9,75.096523,False,0
2,bd358101-d7a1-43b7-b12f-c2af4f092990,QZ993B,FI102Q,invited,2023-05-30 03:55:48.888268+00:00,4.000000,-33.951739,151.180575,Sydney,indoor,...,-33.922051,151.197462,Sydney,2025-05-06 13:09:21.884209+00:00,480,Clear,25.6,80.023992,False,0
3,61eadaa7-579b-437f-aa60-567d248d4625,GC832P,RB514G,maybe,2024-05-15 02:05:11.333627+00:00,817.000000,52.507163,13.432964,Berlin,indoor,...,51.165868,1.764638,London,2025-05-23 09:08:35.832697+00:00,120,Rain,11.3,75.496348,False,1
4,OC899Q,CD621U,LD592D,maybe,2025-04-08 01:40:37.708575+00:00,6.407995,48.843322,2.255623,Paris,indoor,...,48.787649,2.278127,Paris,2025-04-18 13:22:40.369846+00:00,180,Clear,18.9,22.043940,False,1


In [60]:
df.columns

Index(['interaction_id', 'user_id', 'event_id', 'interaction_type',
       'interaction_time', 'interaction_distance_to_event', 'users_user_lat',
       'users_user_lon', 'users_user_city', 'users_user_weather_preference',
       'users_age', 'users_user_interests', 'users_signup_date',
       'users_social_connectedness', 'events_title', 'events_event_type',
       'events_event_lat', 'events_event_lon', 'events_event_city',
       'events_start_time', 'events_duration', 'events_weather_condition',
       'events_temperature', 'events_attendance_rate',
       'events_event_indoor_capability', 'interaction_label'],
      dtype='object')

In [16]:
len(df[df["interaction_label"]==1])

75202

In [21]:
df["events_start_time"].max()

Timestamp('2025-09-22 19:37:51.648789+0000', tz='UTC')

In [15]:
len(df)

165788

In [5]:
data = df.iloc[:1000,:]
data.to_csv("test_data.csv",index=False)

In [1]:
import pandas as pd
interactions_df = pd.read_csv("/home/nkama/masters_thesis_project/interactions.csv")
users_df = pd.read_csv("/home/nkama/masters_thesis_project/users.csv")
events_df = pd.read_csv("/home/nkama/masters_thesis_project/events.csv")
interactions_df.head(2)
interactions_df["interaction_label"] = interactions_df['interaction_type'].apply(
    lambda x: 1 if x in ['maybe', 'invited & maybe', 'yes', 'invited & yes'] else 0
)


In [ ]:
interactions_df.drop(columns="Unnamed: 0", inplace =True)

In [6]:
interactions_df.head(2)

,interaction_id,user_id,event_id,interaction_type,interaction_time,interaction_distance_to_event,interaction_label
0,CJ011J,PG158Y,PP391M,invited & maybe,2025-08-14 19:16:49.777185,10.675627,1
1,NO426K,HL067E,RC216V,invited & yes,2025-06-10 13:20:14.014146,6.735536,1


In [7]:
interactions_df.to_csv("interactions_data.csv")

In [21]:
def preprocess_common(interactions_df, users_df, events_df):
 
    # Create copies of the input dataframes
    interactions_df = interactions_df.copy()
    users_df = users_df.copy()
    events_df = events_df.copy()

    # Drop rows with missing user_id or event_id
    interactions_df = interactions_df.dropna(subset=["user_id", "event_id", "interaction_label"])
    
    # Convert distance_to_event to float
    interactions_df["interaction_distance_to_event"] = interactions_df["interaction_distance_to_event"].fillna(0).astype(float)
    
    # Ensure correct types
    for df in [interactions_df, users_df]:
        df["user_id"] = df["user_id"].astype(str)
    
    for df in [interactions_df, events_df]:
        df["event_id"] = df["event_id"].astype(str)

    interactions_df["interaction_label"] = interactions_df["interaction_label"].astype(int)

    # Convert to string type for TF-IDF fields
    events_df["title"] = events_df["title"].fillna("").astype(str)
    users_df["user_interests"] = users_df["user_interests"].fillna("").astype(str)
    users_df["age"] = users_df["age"].fillna(0).astype(float)

    # Ensure all numeric fields are float - fixed to reference events_df instead of df
    numeric_cols = ["duration", "temperature", "attendance_rate"]
    for col in numeric_cols:
        events_df[col] = pd.to_numeric(events_df[col], errors="coerce").fillna(0).astype(float)

    # Return all three dataframes
    return interactions_df, users_df, events_df

interactions_df, users_df, events_df = preprocess_common(interactions_df, users_df, events_df)
len(interactions_df) + len(users_df) + len(events_df)
merged_df = interactions_df.merge(events_df,on="event_id")\
    .merge(users_df, on="user_id")

merged_df.head()
                                  

merged_df = merged_df.drop(columns=["Unnamed: 0_x", "Unnamed: 0_y", "Unnamed: 0"])
merged_df.head(2)


merged_df = merged_df.rename(columns={"interaction_distance_to_event": "distance_to_event"})
merged_df.head(2)
merged_df.to_csv("merged_interaction_df.csv")


In [31]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(merged_df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [53]:

#Second version

import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset as LFMData
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score, average_precision_score, ndcg_score

# Load data
df = pd.read_csv("merged_interaction_df.csv")
df["interaction_label"] = df["interaction_label"].astype(int)

# Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["interaction_label"])

# --------------------------
# Build user/item metadata as feature dicts
# --------------------------

def extract_user_features(df):
    tfidf = TfidfVectorizer(max_features=50)
    tfidf_matrix = tfidf.fit_transform(df["user_interests"].fillna(""))
    interest_features = tfidf.get_feature_names_out()
    
    user_features = {}
    for idx, row in df.iterrows():
        feats = [f"age:{int(row['age']//10)*10}", f"weather_pref:{row['user_weather_preference']}", f"social:{int(row['social_connectedness']//5)*5}"]
        tfidf_feats = tfidf_matrix[idx].toarray().flatten()
        feats += [f"interest:{interest_features[i]}" for i in tfidf_feats.nonzero()[0]]
        user_features[row["user_id"]] = feats
    return user_features

def extract_item_features(df):
    tfidf = TfidfVectorizer(max_features=50)
    tfidf_matrix = tfidf.fit_transform(df["title"].fillna(""))
    title_features = tfidf.get_feature_names_out()
    
    item_features = {}
    for idx, row in df.iterrows():
        feats = [
            f"event_type:{row['event_type']}",
            f"indoor:{row['event_indoor_capability']}",
            f"temperature:{int(row['temperature']//5)*5}",
            f"duration:{int(row['duration']//60)}h",
        ]
        tfidf_feats = tfidf_matrix[idx].toarray().flatten()
        feats += [f"title:{title_features[i]}" for i in tfidf_feats.nonzero()[0]]
        item_features[row["event_id"]] = feats
    return item_features

user_features_dict = extract_user_features(df)
item_features_dict = extract_item_features(df)

# --------------------------
# Build LightFM Dataset
# --------------------------
dataset = LFMData()
dataset.fit(
    users=df["user_id"].unique(),
    items=df["event_id"].unique(),
    user_features={f for feats in user_features_dict.values() for f in feats},
    item_features={f for feats in item_features_dict.values() for f in feats}
)

# Interactions
interactions, _ = dataset.build_interactions([
    (row["user_id"], row["event_id"], row["interaction_label"]) for _, row in train_df.iterrows()
])

# Feature matrices
user_features = dataset.build_user_features([(uid, feats) for uid, feats in user_features_dict.items()])
item_features = dataset.build_item_features([(iid, feats) for iid, feats in item_features_dict.items()])

# --------------------------
# Train LightFM
# --------------------------
model = LightFM(loss='warp', random_state=42)
model.fit(interactions, user_features=user_features, item_features=item_features, epochs=10, num_threads=4)

# --------------------------
# Evaluate
# --------------------------
# Map IDs
user_map, _, item_map, _ = dataset.mapping()
test_df = test_df[test_df["user_id"].isin(user_map) & test_df["event_id"].isin(item_map)]
test_df["user_idx"] = test_df["user_id"].map(user_map)
test_df["item_idx"] = test_df["event_id"].map(item_map)

# Predict
user_ids = test_df["user_idx"].values
item_ids = test_df["item_idx"].values

test_df["lightfm_score"] = model.predict(
    user_ids, item_ids,
    user_features=user_features,
    item_features=item_features
)


# --------------------------
# Ranking Evaluation
# --------------------------
def evaluate_all(y_true, y_score, k_values=[5, 10]):
    results = {
        "AUC": roc_auc_score(y_true, y_score),
        "MAP": average_precision_score(y_true, y_score)
    }

    y_true = np.array(y_true)
    y_score = np.array(y_score)
    for k in k_values:
        sorted_idx = np.argsort(y_score)[::-1][:k]
        top_k_true = y_true[sorted_idx]
        precision = np.mean(top_k_true)
        recall = np.sum(top_k_true) / np.sum(y_true)
        ndcg = ndcg_score([y_true], [y_score], k=k)
        results[f"Precision@{k}"] = precision
        results[f"Recall@{k}"] = recall
        results[f"NDCG@{k}"] = ndcg
    return results

metrics = evaluate_all(test_df["interaction_label"], test_df["lightfm_score"])
print("📊 LightFM with Metadata Results:")
for metric, val in metrics.items():
    print(f"{metric}: {val:.4f}")


📊 LightFM with Metadata Results:
AUC: 0.3800
MAP: 0.3858
Precision@5: 0.2000
Recall@5: 0.0001
NDCG@5: 0.1461
Precision@10: 0.2000
Recall@10: 0.0001
NDCG@10: 0.1732


In [24]:

# # Enhanced benchmarking evaluation and training pipeline with complete metrics and weather features included
# from sklearn.metrics import roc_auc_score, average_precision_score
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import classification_report
# from sklearn.model_selection import train_test_split
# from surprise import Dataset, Reader, SVD
# from lightfm import LightFM
# from lightfm.data import Dataset as LFMData
# from lightfm.evaluation import precision_at_k, auc_score
# from scipy.sparse import hstack
# import pandas as pd
# import numpy as np
# from catboost import CatBoostClassifier

# # --- Shared Evaluation ---
# def compute_ranking_metrics(y_true, y_score, k=10):
#     # Sort indices by descending score
#     sorted_indices = np.argsort(y_score)[::-1]
#     # Get top-k true labels
#     top_k = np.array(y_true)[sorted_indices][:k]
#     # Calculate precision
#     precision = np.mean(top_k)
#     # Calculate recall
#     recall = np.sum(top_k) / np.sum(y_true) if np.sum(y_true) > 0 else 0
#     # Calculate DCG
#     dcg = np.sum(top_k / np.log2(np.arange(2, len(top_k) + 2)))
#     # Calculate ideal DCG
#     ideal_k = min(int(np.sum(y_true)), k)
#     idcg = np.sum([1 / np.log2(i + 2) for i in range(ideal_k)])
#     # Calculate NDCG
#     ndcg = dcg / idcg if idcg > 0 else 0
#     return precision, recall, ndcg

# def compute_all_metrics(y_true, y_score):
#     # Ensure there are enough positive examples for AUC calculation
#     if len(np.unique(y_true)) < 2:
#         metrics = {
#             "AUC": np.nan,
#             "MAP": np.nan
#         }
#     else:
#         metrics = {
#             "AUC": roc_auc_score(y_true, y_score),
#             "MAP": average_precision_score(y_true, y_score)
#         }
    
#     for k in [5, 10]:
#         p, r, n = compute_ranking_metrics(y_true, y_score, k)
#         metrics[f"Precision@{k}"] = p
#         metrics[f"Recall@{k}"] = r
#         metrics[f"NDCG@{k}"] = n
#     return metrics

# # --- Content-Based Model ---
# def train_content_model(df):
#     # Handle potential NaN values properly
#     df_clean = df.copy()
#     df_clean["title"] = df_clean["title"].fillna("")
#     df_clean["user_interests"] = df_clean["user_interests"].fillna("")

#     # TF-IDF features
#     tfidf_title = TfidfVectorizer(max_features=100)
#     tfidf_interests = TfidfVectorizer(max_features=100)
#     tfidf_title_mat = tfidf_title.fit_transform(df_clean["title"])
#     tfidf_interests_mat = tfidf_interests.fit_transform(df_clean["user_interests"])

#     # Numeric features
#     numeric_cols = ["distance_to_event", "duration", "temperature", "age", "attendance_rate"]
#     df_clean[numeric_cols] = df_clean[numeric_cols].fillna(0)
#     X_numeric = StandardScaler().fit_transform(df_clean[numeric_cols])

#     # Combine all features
#     X = hstack([tfidf_title_mat, tfidf_interests_mat, X_numeric]).toarray()
#     y = df_clean["interaction_label"].astype(int)

#     # Train CatBoost
#     model = CatBoostClassifier(
#         iterations=200,
#         depth=6,
#         learning_rate=0.1,
#         loss_function='Logloss',
#         verbose=False
#     )
#     model.fit(X, y)
#     scores = model.predict_proba(X)[:, 1]

#     return model, compute_all_metrics(y, scores)

# # --- SVD Model ---
# def train_svd(df):
#     # Prepare data for SVD
#     reader = Reader(rating_scale=(0, 1))
#     df_svd = df[["user_id", "event_id", "interaction_label"]].copy()
    
#     # Convert all to strings to ensure compatibility
#     df_svd["user_id"] = df_svd["user_id"].astype(str)
#     df_svd["event_id"] = df_svd["event_id"].astype(str)
#     df_svd["interaction_label"] = df_svd["interaction_label"].astype(float)
    
#     data = Dataset.load_from_df(df_svd, reader)
#     trainset = data.build_full_trainset()
    
#     # Train SVD model
#     svd = SVD(n_epochs=20).fit(trainset)
    
#     # Generate predictions for all user-item pairs
#     df_copy = df.copy()
#     df_copy["svd_score"] = df_copy.apply(
#         lambda row: svd.predict(str(row["user_id"]), str(row["event_id"])).est, 
#         axis=1
#     )
    
#     return svd, compute_all_metrics(df_copy["interaction_label"].astype(int), df_copy["svd_score"])


# # --- Final Hybrid Training with Train/Test Separation ---
# def hybrid_model(train_df, val_df):
#     # Step 1: Train SVD model on train_df
#     reader = Reader(rating_scale=(0, 1))
#     train_svd_data = Dataset.load_from_df(train_df[["user_id", "event_id", "interaction_label"]].astype(str), reader)
#     trainset = train_svd_data.build_full_trainset()
#     svd_model = SVD(n_epochs=20).fit(trainset)

#     # Step 2: Compute SVD scores for both train and val
#     train_df = train_df.copy()
#     val_df = val_df.copy()
#     train_df["svd_score"] = train_df.apply(
#         lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
#     )
#     val_df["svd_score"] = val_df.apply(
#         lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
#     )

#     # Step 3: TF-IDF + numeric features
#     tfidf_title = TfidfVectorizer(max_features=50)
#     tfidf_interests = TfidfVectorizer(max_features=50)
#     tfidf_title.fit(train_df["title"].fillna(""))
#     tfidf_interests.fit(train_df["user_interests"].fillna(""))

#     X_train_text = hstack([
#         tfidf_title.transform(train_df["title"].fillna("")),
#         tfidf_interests.transform(train_df["user_interests"].fillna(""))
#     ])
#     X_val_text = hstack([
#         tfidf_title.transform(val_df["title"].fillna("")),
#         tfidf_interests.transform(val_df["user_interests"].fillna(""))
#     ])

#     numeric_cols = ["distance_to_event", "duration", "temperature", "age", "attendance_rate", "svd_score"]
#     train_df[numeric_cols] = train_df[numeric_cols].fillna(0)
#     val_df[numeric_cols] = val_df[numeric_cols].fillna(0)

#     scaler = StandardScaler().fit(train_df[numeric_cols])
#     X_train_numeric = scaler.transform(train_df[numeric_cols])
#     X_val_numeric = scaler.transform(val_df[numeric_cols])

#     # Combine all features
#     X_train = hstack([X_train_text, X_train_numeric]).toarray()
#     X_val = hstack([X_val_text, X_val_numeric]).toarray()
#     y_train = train_df["interaction_label"].astype(int)
#     y_val = val_df["interaction_label"].astype(int)

#     model = CatBoostClassifier(
#         iterations=200,
#         depth=6,
#         learning_rate=0.1,
#         loss_function='Logloss',
#         verbose=False
#     )
#     model.fit(X_train, y_train)

#     scores = model.predict_proba(X_val)[:, 1]
#     return model, compute_all_metrics(y_val, scores)

# "✅ Hybrid model updated to use CatBoost for better non-linear modeling."

# # Final all-model benchmark comparison with strict train/validation split and consistent evaluation
# def compare_all_models_strict(train_df, val_df):
#     train_df = train_df.copy()
#     val_df = val_df.copy()

#     # Ensure correct types and fill missing
#     for df in [train_df, val_df]:
#         df["user_id"] = df["user_id"].astype(str)
#         df["event_id"] = df["event_id"].astype(str)
#         df["interaction_label"] = df["interaction_label"].astype(int)
#         df["title"] = df["title"].fillna("").astype(str)
#         df["user_interests"] = df["user_interests"].fillna("").astype(str)

#     print("Training Content-Based model...")
#     _, content_scores = train_content_model(train_df)

#     print("Training SVD model...")
#     _, svd_scores = train_svd(train_df)

#     print("Training Hybrid (SVD + Content + Weather) model with strict validation...")
#     _, hybrid_scores = hybrid_model(train_df, val_df)

  

#     # Assemble final results
#     results_df = pd.DataFrame({
#         "Content-Based": content_scores,
#         "SVD": svd_scores,
#         "Hybrid": hybrid_scores,

#     })

#     return results_df.T

# "✅ All-model benchmark comparison function finalized with strict train/val split and metric evaluation."

# from sklearn.model_selection import train_test_split

# train_df, val_df = train_test_split(merged_df, test_size=0.2, random_state=42)

# benchmark_results = compare_all_models_strict(train_df, val_df)
# display(benchmark_results)


Training Content-Based model...
Training SVD model...
Training Hybrid (SVD + Content + Weather) model with strict validation...


,AUC,MAP,Precision@5,Recall@5,NDCG@5,Precision@10,Recall@10,NDCG@10
Content-Based,0.735287,0.712871,1.0,0.000083,1.0,1.0,0.000165,1.000000
SVD,0.910474,0.906715,1.0,0.000083,1.0,1.0,0.000165,1.000000
Hybrid,0.591013,0.572386,1.0,0.000329,1.0,0.9,0.000591,0.921602


In [50]:
def train_content_model(train_df, val_df):
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    
    # Clean and prepare training data
    train_clean = train_df.copy()
    train_clean["title"] = train_clean["title"].fillna("")
    train_clean["user_interests"] = train_clean["user_interests"].fillna("")
    
    # Clean and prepare validation data
    val_clean = val_df.copy()
    val_clean["title"] = val_clean["title"].fillna("")
    val_clean["user_interests"] = val_clean["user_interests"].fillna("")

    # TF-IDF features - fit only on training data
    tfidf_title = TfidfVectorizer(max_features=100)
    tfidf_interests = TfidfVectorizer(max_features=100)
    tfidf_title_mat_train = tfidf_title.fit_transform(train_clean["title"])
    tfidf_interests_mat_train = tfidf_interests.fit_transform(train_clean["user_interests"])
    
    # Transform validation data using fitted vectorizers
    tfidf_title_mat_val = tfidf_title.transform(val_clean["title"])
    tfidf_interests_mat_val = tfidf_interests.transform(val_clean["user_interests"])

    # Numeric features
    numeric_cols = ["distance_to_event", "duration", "temperature", "age", "attendance_rate"]
    train_clean[numeric_cols] = train_clean[numeric_cols].fillna(0)
    val_clean[numeric_cols] = val_clean[numeric_cols].fillna(0)
    
    # Fit scaler on training data only
    scaler = StandardScaler().fit(train_clean[numeric_cols])
    X_numeric_train = scaler.transform(train_clean[numeric_cols])
    X_numeric_val = scaler.transform(val_clean[numeric_cols])
    
    # Categorical features with OneHotEncoder
    categorical_cols = ['events_weather_condition', 'events_event_indoor_capability', 'users_user_weather_preference']
    
    # Fill missing values in categorical columns
    for col in categorical_cols:
        train_clean[col] = train_clean[col].fillna('unknown')
        val_clean[col] = val_clean[col].fillna('unknown')
    
    # Create OneHotEncoder for categorical features
    encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    X_cat_train = encoder.fit_transform(train_clean[categorical_cols])
    X_cat_val = encoder.transform(val_clean[categorical_cols])

    # Combine all features
    X_train = hstack([tfidf_title_mat_train, tfidf_interests_mat_train, X_numeric_train, X_cat_train]).toarray()
    X_val = hstack([tfidf_title_mat_val, tfidf_interests_mat_val, X_numeric_val, X_cat_val]).toarray()
    
    y_train = train_clean["interaction_label"].astype(int)
    y_val = val_clean["interaction_label"].astype(int)

    # Train CatBoost on training data only
    model = CatBoostClassifier(
        iterations=200,
        depth=6,
        learning_rate=0.1,
        loss_function='Logloss',
        verbose=False
    )
    model.fit(X_train, y_train)
    
    # Predict on validation data
    val_scores = model.predict_proba(X_val)[:, 1]

    return model, compute_all_metrics(y_val, val_scores)

In [43]:
def train_svd(train_df, val_df):
    # Prepare training data for SVD
    reader = Reader(rating_scale=(0, 1))
    train_svd = train_df[["user_id", "event_id", "interaction_label"]].copy()
    
    # Convert all to strings to ensure compatibility
    train_svd["user_id"] = train_svd["user_id"].astype(str)
    train_svd["event_id"] = train_svd["event_id"].astype(str)
    train_svd["interaction_label"] = train_svd["interaction_label"].astype(float)
    
    data = Dataset.load_from_df(train_svd, reader)
    trainset = data.build_full_trainset()
    
    # Train SVD model on training data only
    svd = SVD(n_epochs=50).fit(trainset)
    
    # Generate predictions for validation data
    val_copy = val_df.copy()
    val_copy["svd_score"] = val_copy.apply(
        lambda row: svd.predict(str(row["user_id"]), str(row["event_id"])).est, 
        axis=1
    )
    
    return svd, compute_all_metrics(val_copy["interaction_label"].astype(int), val_copy["svd_score"])

In [51]:
def hybrid_model(train_df, val_df):
    from sklearn.preprocessing import OneHotEncoder
    
    # Step 1: Train SVD model on train_df
    reader = Reader(rating_scale=(0, 1))
    train_svd_data = Dataset.load_from_df(train_df[["user_id", "event_id", "interaction_label"]].astype(str), reader)
    trainset = train_svd_data.build_full_trainset()
    svd_model = SVD(n_epochs=20).fit(trainset)

    # Step 2: Compute SVD scores for both train and val
    train_df = train_df.copy()
    val_df = val_df.copy()
    train_df["svd_score"] = train_df.apply(
        lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
    )
    val_df["svd_score"] = val_df.apply(
        lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
    )

    # Step 3: TF-IDF + numeric features
    tfidf_title = TfidfVectorizer(max_features=50)
    tfidf_interests = TfidfVectorizer(max_features=50)
    tfidf_title.fit(train_df["title"].fillna(""))
    tfidf_interests.fit(train_df["user_interests"].fillna(""))

    X_train_text = hstack([
        tfidf_title.transform(train_df["title"].fillna("")),
        tfidf_interests.transform(train_df["user_interests"].fillna(""))
    ])
    X_val_text = hstack([
        tfidf_title.transform(val_df["title"].fillna("")),
        tfidf_interests.transform(val_df["user_interests"].fillna(""))
    ])

    # Numeric features including SVD score
    numeric_cols = ["distance_to_event", "duration", "temperature", "age", "attendance_rate", "svd_score"]
    train_df[numeric_cols] = train_df[numeric_cols].fillna(0)
    val_df[numeric_cols] = val_df[numeric_cols].fillna(0)

    scaler = StandardScaler().fit(train_df[numeric_cols])
    X_train_numeric = scaler.transform(train_df[numeric_cols])
    X_val_numeric = scaler.transform(val_df[numeric_cols])
    
    # Categorical features with OneHotEncoder
    categorical_cols = ['events_weather_condition', 'events_event_indoor_capability', 'users_user_weather_preference']
    
    # Fill missing values in categorical columns
    for col in categorical_cols:
        train_df[col] = train_df[col].fillna('unknown')
        val_df[col] = val_df[col].fillna('unknown')
    
    # Create OneHotEncoder for categorical features
    encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    X_cat_train = encoder.fit_transform(train_df[categorical_cols])
    X_cat_val = encoder.transform(val_df[categorical_cols])

    # Combine all features
    X_train = hstack([X_train_text, X_train_numeric, X_cat_train]).toarray()
    X_val = hstack([X_val_text, X_val_numeric, X_cat_val]).toarray()
    
    y_train = train_df["interaction_label"].astype(int)
    y_val = val_df["interaction_label"].astype(int)

    model = CatBoostClassifier(
        iterations=200,
        depth=6,
        learning_rate=0.1,
        loss_function='Logloss',
        verbose=False
    )
    model.fit(X_train, y_train)

    scores = model.predict_proba(X_val)[:, 1]
    return model, compute_all_metrics(y_val, scores)
'✅ Hybrid model updated to use CatBoost for better non-linear modeling.'

'✅ Hybrid model updated to use CatBoost for better non-linear modeling.'

In [47]:
def compare_all_models_strict(train_df, val_df):
    train_df = train_df.copy()
    val_df = val_df.copy()

    # Ensure correct types and fill missing
    for df in [train_df, val_df]:
        df["user_id"] = df["user_id"].astype(str)
        df["event_id"] = df["event_id"].astype(str)
        df["interaction_label"] = df["interaction_label"].astype(int)
        df["title"] = df["title"].fillna("").astype(str)
        df["user_interests"] = df["user_interests"].fillna("").astype(str)
        
        # Ensure categorical columns exist
        for col in ['events_weather_condition', 'events_event_indoor_capability', 'users_user_weather_preference']:
            if col not in df.columns:
                df[col] = "unknown"

    print("Training Content-Based model...")
    _, content_scores = train_content_model(train_df, val_df)

    print("Training SVD model...")
    _, svd_scores = train_svd(train_df, val_df)

    print("Training Hybrid (SVD + Content + Weather) model with strict validation...")
    _, hybrid_scores = hybrid_model(train_df, val_df)

    # Assemble final results
    results_df = pd.DataFrame({
        "Content-Based": content_scores,
        "SVD": svd_scores,
        "Hybrid": hybrid_scores,
    })

    return results_df.T

In [52]:
compare_all_models_strict(train_df, val_df)

Training Content-Based model...
Training SVD model...
Training Hybrid (SVD + Content + Weather) model with strict validation...


,AUC,MAP,Precision@5,Recall@5,NDCG@5,Precision@10,Recall@10,NDCG@10
Content-Based,0.703276,0.670024,1.0,0.000329,1.000000,0.9,0.000591,0.933746
SVD,0.622998,0.576177,1.0,0.000329,1.000000,0.8,0.000526,0.870125
Hybrid,0.592051,0.570906,0.6,0.000197,0.654809,0.7,0.000460,0.709740


Weather Model Scores:
AUC: 0.7011
Average Precision (MAP): 0.6712
Precision@5: 0.8000
Recall@5: 0.0005
NDCG@5: 0.9344
Precision@10: 0.8000
Recall@10: 0.0011
NDCG@10: 0.8546

No-Weather Model Scores:
AUC: 0.7064
Average Precision (MAP): 0.6755
Precision@5: 0.8000
Recall@5: 0.0005
NDCG@5: 0.6608
Precision@10: 0.9000
Recall@10: 0.0012
NDCG@10: 0.7799

In [81]:
from lightfm import LightFM
from lightfm.data import Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

def prepare_lightfm_data(df):
    # Create TF-IDF models
    user_tfidf = TfidfVectorizer(max_features=100)
    item_tfidf = TfidfVectorizer(max_features=100)
    
    user_tfidf.fit(df['users_user_interests'].fillna(''))
    item_tfidf.fit((df['events_title'] + ' ' + df['events_event_type']).fillna(''))
    
    dataset = Dataset()
    dataset.fit(
        users=df['user_id'].unique(),
        items=df['event_id'].unique(),
        user_features=user_tfidf.get_feature_names_out().tolist(),
        item_features=item_tfidf.get_feature_names_out().tolist()
    )
    
    return dataset, user_tfidf, item_tfidf

def train_lightfm_model(df):
    dataset, user_tfidf, item_tfidf = prepare_lightfm_data(df)
    train, test = train_test_split(df, test_size=0.2, random_state=42)

    # Build user content tuples
    user_features_matrix = user_tfidf.transform(train['users_user_interests'].fillna(''))
    user_features_data = [
        (uid, user_tfidf.get_feature_names_out()[user_features_matrix[i].nonzero()[1]].tolist())
        for i, uid in enumerate(train['user_id'])
    ]

    # Build item content tuples
    item_texts = (train['events_title'] + ' ' + train['events_event_type']).fillna('')
    item_features_matrix = item_tfidf.transform(item_texts)
    item_features_data = [
        (eid, item_tfidf.get_feature_names_out()[item_features_matrix[i].nonzero()[1]].tolist())
        for i, eid in enumerate(train['event_id'])
    ]

    # Build features
    user_features = dataset.build_user_features(user_features_data)
    item_features = dataset.build_item_features(item_features_data)

    # Build interactions
    interactions, _ = dataset.build_interactions([
        (row['user_id'], row['event_id'])
        for _, row in train.iterrows()
    ])

    # Train model
    model = LightFM(loss='warp')
    model.fit(
        interactions,
        user_features=user_features,
        item_features=item_features,
        epochs=20
    )

    # Filter test for known users/items
    known_users = dict(dataset.mapping()[0])
    known_items = dict(dataset.mapping()[2])
    test_filtered = test[
        test['user_id'].isin(known_users) & test['event_id'].isin(known_items)
    ]

    if not test_filtered.empty:
        user_ids = test_filtered['user_id'].map(known_users).values
        item_ids = test_filtered['event_id'].map(known_items).values

        scores = model.predict(
            user_ids,
            item_ids,
            user_features=user_features,
            item_features=item_features
        )

        return {
            'model': model,
            'metrics': compute_all_metrics(test_filtered['interaction_label'].values, scores)
        }

    print("⚠️ No valid test samples after filtering.")
    return {'model': model, 'metrics': {}}




In [83]:
scores = train_lightfm_model(df)

In [84]:
scores["metrics"]

{'AUC': 0.3519315613644091,
 'MAP': 0.36895397602797764,
 'Precision@5': 0.0,
 'Recall@5': 0.0,
 'NDCG@5': 0.0,
 'Precision@10': 0.1,
 'Recall@10': 6.676904587033452e-05,
 'NDCG@10': 0.06362078819895173}

: 